# Helpfulness Prediction With Random Forest
## Data Science and Big Data Analytics Project

---

### Authors: 
- **Andrea Alberti** ([GitHub](https://github.com/AndreaAlberti07))
- **Davide Ligari** ([GitHub](https://github.com/DavideLigari01))
- **Cristian Andreoli** ([GitHub](https://github.com/CristianAndreoli94))

### Date: September 2023

---

## Data: 
The chosen dataset is [Amazon Books Reviews](https://www.kaggle.com/datasets/mohamedbakhet/amazon-books-reviews).


## Goal:
Build a model able to predict the helpfulness of a review based on its content. 

---

### Model Training and Evaluation using Grid Search Cross Validation

The features are extracted from the review text using Word2Vec with window = 5, creating vectors of 30 features for each word. The document is the average of the vectors of the words that compose it. The model used is a Random Forest Classifier.

| N_grid | n_estimators | max_depth | min_samples_split | Test MSE             | Test $R^2$         |
| ------ | ------------ | --------- | ----------------- | -------------------- | ------------------ |
| 2      | 100          | None      | 4                 | 0.025927002564121476 | 0.2581554095121458 |
| 3      | 200          | None      | 2                 | 0.025702599368344427 | 0.2531769186466868 |


In [ ]:
import joblib
import numpy as np
import pymongo as pm
import pandas as pd
import numpy as np
import gensim
import sklearn as sk
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error,r2_score


In [ ]:
### 1. Data Loading
client = pm.MongoClient('mongodb://localhost:27017/')
spark_db = client['spark_db']
books_ratings = spark_db['book_reviews']

In [3]:

# Load the training and test data
train_data = np.load(
    '../_gitignore/train_data_wv2_30_5.npz', allow_pickle=True)
test_data = np.load(
    '../_gitignore/test_data_w2v_30_5.npz', allow_pickle=True)
X_train_embedding = train_data['x']
Y_train = train_data['y']
X_test_embedding = test_data['x']
Y_test = test_data['y']

# Define the hyperparameter grid you want to search over
param_grid_1 = {
    'n_estimators': [100, 200],       # Number of trees in the forest
    'max_depth': [30, 50],     # Maximum depth of the trees
    'min_samples_split': [4, 8],    # Minimum samples required to split a node
    # Add more hyperparameters and their values to explore here
}

param_grid_2 = {
    'n_estimators': [50, 100],       # Number of trees in the forest
    'max_depth': [None, 10],     # Maximum depth of the trees
    'min_samples_split': [4, 8],    # Minimum samples required to split a node
    # Add more hyperparameters and their values to explore here
}


param_grid_3 = {
    'n_estimators': [50, 100,200],       # Number of trees in the forest
    'max_depth': [None, 10],     # Maximum depth of the trees
    'min_samples_split': [2, 5],    # Minimum samples required to split a node
    # Add more hyperparameters and their values to explore here
}
# Create the RandomForestRegressor model
rand_forest = RandomForestRegressor(random_state=42)

# Create a GridSearchCV object with the model and hyperparameter grid
grid_search = GridSearchCV(estimator=rand_forest,
                           param_grid=param_grid_3, cv=2, n_jobs=-1)

# Fit the GridSearchCV object to your training data
grid_search.fit(X_train_embedding, Y_train)

# Get the best hyperparameters and the best estimator
best_params = grid_search.best_params_
best_estimator = grid_search.best_estimator_

# Save the best model to a file
joblib.dump(best_estimator,
            f'../trained/rand_forest_model_md_mss_ne.gz', compress=('gzip', 3))

# Print the best hyperparameters
print("Best Hyperparameters:", best_params)

# Evaluate the best estimator on the test set
Y_test_pred = best_estimator.predict(X_test_embedding)
mse = mean_squared_error(Y_test, Y_test_pred)
print("Mean Squared Error on Test Set:", mse)

Y_train_pred = best_estimator.predict(X_train_embedding)
mse = mean_squared_error(Y_train, Y_train_pred)
print("Mean Squared Error on Train Set:", mse)

Best Hyperparameters: {'max_depth': None, 'min_samples_split': 2, 'n_estimators': 100}
Mean Squared Error on Test Set: 0.02587508853631114
Mean Squared Error on Train Set: 0.0036453911772139323


In [5]:
# model_path = '../trained/rand_forest_model_mdNone_mss4_ne100.gz'
model_path = '../trained/rand_forest_model_mdNone_mss2_ne100.gz'
loaded_model = joblib.load(model_path)
Y_test_pred = best_estimator.predict(X_test_embedding)

r2= r2_score(Y_test, Y_test_pred)
print('Model mdNone_mss4_ne100')
print("R2 Score on Test Set:", r2)

Y_train_pred = best_estimator.predict(X_train_embedding)
r2= r2_score(Y_train, Y_train_pred)
print("R2 Score on Train Set:", r2)

Model mdNone_mss4_ne100
R2 Score on Test Set: 0.2531769186466868
R2 Score on Train Set: 0.8941849152318941


---

## Use Total votes as feature to predict helpful votes

### Not useful since Total votes are not available at the time of the review

In [8]:
pipeline_remove = {'$match':{
                        'review/score':{'$exists':True},
                        'N_helpful'	:{'$exists':True, '$ne':0},
                        'Tot_votes'	:{'$exists':True, '$ne':0}
                        }
    
                }

smoothing_param = 1

pipeline_project = {'$project':{
                            'review/text':1,
                            'Tot_votes':1,
                            'N_helpful':1,
                            '_id':0,
                            }
                    }

mongo_dataset = books_ratings.aggregate([pipeline_remove, pipeline_project])
df_dataset = pd.DataFrame(list(mongo_dataset))
arr_dataset = np.array(df_dataset)

X_train, X_test, Y_train, Y_test = sk.model_selection.train_test_split(arr_dataset[:,0:2], arr_dataset[:,2], test_size=0.2, random_state=42)

In [9]:
stop_words = set(stopwords.words('english'))

def preprocess(doc):
    tokens = gensim.utils.simple_preprocess(doc)
    return [token for token in tokens if token not in stop_words]

X_train_w2v = [preprocess(doc[0]) for doc in X_train[:]]

In [10]:
model = gensim.models.Word2Vec(X_train_w2v, vector_size=30, window=5, min_count=2)

def get_embedding(doc):
    embeddings = []
    words = preprocess(doc[0])
    for word in words:
        if word in model.wv:
            embeddings.append(np.append(model.wv[word], doc[1]))
    if len(embeddings) > 0:
         return np.mean(embeddings, axis=0)
    else:
        return np.zeros(model.vector_size+1)

X_train_embedding = [get_embedding(doc) for doc in X_train[:]]
X_test_embedding = [get_embedding(doc) for doc in X_test[:]]

In [11]:
np.savez('../_gitignore/train_data_wv2_totvotes_30_5.npz',x = X_train_embedding, y = Y_train)
np.savez('../_gitignore/test_data_w2v_totvotes_30_5.npz',x = X_test_embedding, y = Y_test)

In [12]:
rand_forest = RandomForestRegressor(n_estimators=100, random_state=42)
rand_forest.fit(X_train_embedding, Y_train)
joblib.dump(rand_forest, '../model/rand_forest_model_totvotes.gz', compress=('gzip', 3))

['../model/rand_forest_model_totvotes.gz']

In [13]:
Y_test_pred = rand_forest.predict(X_test_embedding)
Y_train_pred = rand_forest.predict(X_train_embedding)

In [15]:
def rmse(Y_test, Y_pred):
    return np.sqrt(sk.metrics.mean_squared_error(Y_test, Y_pred))

rmse_test = rmse(Y_test, Y_test_pred)
print('RMSE with smoothing test: ', rmse_test)

rmse_train = rmse(Y_train, Y_train_pred)
print('RMSE with smoothing train: ', rmse_train)

RMSE with smoothing test:  9.037798949817295
RMSE with smoothing train:  3.023416276713991
